In [ ]:
from transformers import (
    LongformerForMaskedLM,
    LongformerTokenizer,
    LongformerConfig,
    Trainer,
    DataCollatorForLanguageModeling,
    TrainingArguments,
)
import torch
import torch.nn as nn

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from functools import partial

In [ ]:
model = AutoModelForMaskedLM.from_pretrained('allenai/longformer-base-4096')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096')

In [ ]:
model.save_pretrained("MaskedLM/longformer")

In [ ]:
tokenizer.save_pretrained("MaskedLM/longformer")

In [ ]:
import transformers
tokenizer = AutoTokenizer.from_pretrained('MaskedLM/longformer')

In [ ]:
df2 = pd.read_pickle("df2.pickle")

In [ ]:
for i in ["{{PHONE}}","{{UK_MOBILE}}","{{EMAIL}}"]:
    df2.message = df2.message.str.replace(i,"[scrubbed]",regex=False)


In [ ]:
bot_messages = df2[df2.interaction == "bot"]
vc=bot_messages.message.value_counts()
for k,v in vc.iteritems():
    if v<1000:
        df2.loc[bot_messages[bot_messages.message == k].index, "message"] = tokenizer.unk_token

In [ ]:
tokenizer.add_tokens(list(vc[vc>=1000].keys()))

In [ ]:
df2['message'] = df2.message.str.strip().apply(lambda a:" "+a)
df2['encoded_message'] = "[" + df2.interaction + "]" + df2.message
encoded_conversations = df2.groupby('conversation_id').encoded_message.agg(" ".join).apply(
    lambda a : f"{tokenizer.bos_token}{a}{tokenizer.eos_token}"
)

In [ ]:
from tqdm.auto import tqdm
import re
tqdm.pandas(desc="find tokens")
token_lambda = lambda a: re.findall("[\[|\{]+\w+[\]|\}]+",a) if type(a)==str else np.nan
a = encoded_conversations.progress_apply(token_lambda)

from collections import defaultdict
freq = defaultdict(lambda : 0)

tqdm.pandas()
for l in tqdm(a):
    for i in l:
        freq[i] +=1

In [ ]:
new_tokens = [i for i,j in freq.items() if j>10000]
print(new_tokens)
tokenizer.add_tokens(new_tokens)

In [ ]:
tokenizer.save_pretrained('MaskedLM/longformer')